In [4]:
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN
from geopy.distance import geodesic
from sklearn.metrics.pairwise import haversine_distances
from math import radians 
from geopy.distance import geodesic

In [3]:
import folium

## EDA

In [2]:
prediction_output = pd.read_excel('../data/business_algo_output/recommendation_output.xlsx')
prediction_output = prediction_output.rename(columns={'FCID': 'StockPointID'})
customer_dim = pd.read_excel('../data/business_algo_output/customer_data.xlsx')
customer_dim = customer_dim.rename(columns={'Longitude': 'Long', 'Latitude': 'Lat'})

In [4]:
# print columns in prediction_output
print(prediction_output.columns.tolist())

# print top 5 rows of prediction_output
prediction_output.head()

['StockPointName', 'StockPointID', 'CustomerID', 'Category', 'SKUID', 'SKUCODE', 'rank_hist_cust_cat', 'L4M_n_orders', 'L4M_tot_qty', 'L4M_AOV', 'L4M_AOQ', 'L4M_Cust_Activity_SKU', 'L4M_Cust_Activity_Day_SKU', 'rank_hist_sp_cat', 'L4M_SP_Activity_SKU', 'rank_survey', 'SuggestType', 'SP_Product_Tag', 'RECOMMEND_FOR_PUSH']


,StockPointName,StockPointID,CustomerID,Category,SKUID,SKUCODE,rank_hist_cust_cat,L4M_n_orders,L4M_tot_qty,L4M_AOV,L4M_AOQ,L4M_Cust_Activity_SKU,L4M_Cust_Activity_Day_SKU,rank_hist_sp_cat,L4M_SP_Activity_SKU,rank_survey,SuggestType,SP_Product_Tag,RECOMMEND_FOR_PUSH
0,OmniHub Alimosho Lagos - Barka-Agro and Genera...,1647081,1770455,Noodles,19679,IOC001,1.0,1.0,15.0,117000.0,15.0,2025-04-05,56.0,1,2025-05-31,3.0,TOP 5 - Customer By Category,Express,Yes
1,OmniHub Alimosho Lagos - Barka-Agro and Genera...,1647081,1770455,Noodles,22026,MIMEE_RC070,1.0,1.0,15.0,117000.0,15.0,2025-05-27,4.0,1,2025-05-31,1.0,TOP 5 - Customer By Category,Express,No
2,OmniHub Alimosho Lagos - Barka-Agro and Genera...,1647081,1770455,Noodles,22028,MIMEE_RC100,1.0,1.0,15.0,117000.0,15.0,NaT,NaN,1,2025-05-31,1.0,TOP 5 - Customer By Category,Express,Yes
3,OmniHub Alimosho Lagos - Barka-Agro and Genera...,1647081,1770455,Noodles,22298,GPNOOD70G,1.0,1.0,15.0,117000.0,15.0,NaT,NaN,1,2025-05-31,1.0,TOP 5 - Customer By Category,Express,Yes
4,OmniHub Alimosho Lagos - Barka-Agro and Genera...,1647081,1770455,Noodles,22462,GPJCNOOD70G,1.0,1.0,15.0,117000.0,15.0,2025-04-17,44.0,1,2025-05-31,2.0,TOP 5 - Customer By Category,Express,Yes


In [8]:
# per StockPointName and RECOMMEND_FOR_PUSH = "Yes", count number of unique CustomerID
stockpoint_recommendation = prediction_output[prediction_output['RECOMMEND_FOR_PUSH'] == 'Yes'].groupby('StockPointName')
stockpoint_recommendation['CustomerID'].nunique().reset_index().rename(columns={'CustomerID': 'UniqueCustomerCount'})

,StockPointName,UniqueCustomerCount
0,OmniHub Alimosho Lagos - Barka-Agro and Genera...,483
1,OmniHub Apapa Lagos - CAUSEWAY,1843


In [13]:
# Unique SKU per SP_Product_Tag 
prediction_output.query('RECOMMEND_FOR_PUSH == "Yes" and SP_Product_Tag != "Standard-Inactive"')\
    .shape[0]
    # (prediction_output['RECOMMEND_FOR_PUSH'] == 'Yes') & ()] \ 
    # .groupby(['StockPointName','SP_Product_Tag'])\
    # ['CustomerID'].nunique().reset_index().rename(columns={'CustomerID': 'UniqueCustomerCount'})

44441

## Clustering

In [11]:
def cluster_customers_dbscan(prediction_output, customer_dim, eps_km=2.0, min_samples=3):
    """
    Cluster customers by location for each stock point using DBSCAN and provide cluster metadata.
    
    Args:
        prediction_output (pd.DataFrame): DataFrame with StockPointID and CustomerID
        customer_dim (pd.DataFrame): DataFrame with CustomerID, Long, and Lat
        eps_km (float): Maximum distance (in kilometers) for points to be in the same cluster
        min_samples (int): Minimum number of points to form a cluster
    
    Returns:
        tuple: (assignment_df, summary_df) containing customer assignments and cluster summaries
    """
    # Merge prediction_output with customer_dim to get coordinates
    merged_data = prediction_output.merge(customer_dim, on='CustomerID')
    
    # Initialize lists to hold results
    assignment_list = []
    summary_list = []
    
    # Get unique stock points
    stock_points = merged_data['StockPointID'].unique()
    
    for sp in stock_points:
        # Filter customers for this stock point
        sp_data = merged_data[merged_data['StockPointID'] == sp]
        customers = sp_data[['CustomerID', 'Long', 'Lat']].copy()
        
        # Number of customers
        n_customers = len(customers)
        
        if n_customers == 0:
            continue  # No customers, skip
        
        # Prepare coordinates for DBSCAN (latitude, longitude in radians)
        coords = customers[['Lat', 'Long']].values
        coords_rad = np.radians(coords)
        
        # Convert eps from kilometers to radians (approximate Earth radius: 6371 km)
        eps_rad = eps_km / 6371.0
        
        # Apply DBSCAN with haversine metric for geodesic distances
        dbscan = DBSCAN(eps=eps_rad, min_samples=min_samples, metric='haversine').fit(coords_rad)
        labels = dbscan.labels_
        
        # Handle case where no clusters are formed (all noise)
        if max(labels) < 0 and n_customers >= min_samples:
            # Try a larger eps to ensure at least one cluster
            eps_rad *= 2
            dbscan = DBSCAN(eps=eps_rad, min_samples=min_samples, metric='haversine').fit(coords_rad)
            labels = dbscan.labels_
        
        # Assign cluster labels to customers
        customers['ClusterID'] = labels
        
        # Handle noise points (ClusterID = -1) by assigning to nearest cluster
        if -1 in labels:
            non_noise = customers[customers['ClusterID'] != -1]
            noise = customers[customers['ClusterID'] == -1]
            if len(non_noise) > 0:
                # Calculate centroids of non-noise clusters
                centroids = non_noise.groupby('ClusterID')[['Lat', 'Long']].mean().values
                noise_coords = noise[['Lat', 'Long']].values
                # Compute distances to centroids
                distances = np.zeros((len(noise), len(centroids)))
                for i, n_coord in enumerate(noise_coords):
                    for j, c_coord in enumerate(centroids):
                        distances[i, j] = geodesic(n_coord, c_coord).kilometers
                # Assign noise points to nearest cluster
                nearest_clusters = np.argmin(distances, axis=1)
                noise['ClusterID'] = nearest_clusters
                customers.update(noise)
        
        # Ensure at least 3 clusters if possible
        unique_clusters = customers[customers['ClusterID'] != -1]['ClusterID'].nunique()
        if unique_clusters < 3 and n_customers >= min_samples * 3:
            # Reduce eps to encourage more clusters
            eps_rad /= 1.5
            dbscan = DBSCAN(eps=eps_rad, min_samples=min_samples, metric='haversine').fit(coords_rad)
            customers['ClusterID'] = dbscan.labels_
            # Reassign noise points if any
            if -1 in customers['ClusterID'].values:
                non_noise = customers[customers['ClusterID'] != -1]
                noise = customers[customers['ClusterID'] == -1]
                if len(non_noise) > 0:
                    centroids = non_noise.groupby('ClusterID')[['Lat', 'Long']].mean().values
                    noise_coords = noise[['Lat', 'Long']].values
                    distances = np.zeros((len(noise), len(centroids)))
                    for i, n_coord in enumerate(noise_coords):
                        for j, c_coord in enumerate(centroids):
                            distances[i, j] = geodesic(n_coord, c_coord).kilometers
                    nearest_clusters = np.argmin(distances, axis=1)
                    noise['ClusterID'] = nearest_clusters
                    customers.update(noise)
        
        # Calculate distances to centroids
        centroids = customers[customers['ClusterID'] != -1].groupby('ClusterID')[['Lat', 'Long']].mean().reset_index()
        distances = []
        for i, row in customers.iterrows():
            if row['ClusterID'] == -1:
                distances.append(0)  # Should not occur after noise assignment
            else:
                customer_coord = (row['Lat'], row['Long'])
                centroid_coord = centroids[centroids['ClusterID'] == row['ClusterID']][['Lat', 'Long']].values[0]
                distance = geodesic(customer_coord, centroid_coord).kilometers
                distances.append(distance)
        
        customers['DistanceToCentroid'] = distances
        
        # Add StockPointID to the assignment table
        customers['StockPointID'] = sp
        
        # Select relevant columns for the assignment table
        assignment = customers[['StockPointID', 'CustomerID', 'ClusterID', 'DistanceToCentroid']]
        assignment_list.append(assignment)
        
        # Create summary table for each cluster
        summary = customers.groupby('ClusterID').agg(
            NumCustomers=('CustomerID', 'count'),
            AvgDistance=('DistanceToCentroid', 'mean')
        ).reset_index()
        
        # Add centroid coordinates to the summary table
        if not centroids.empty:
            summary = summary.merge(centroids[['ClusterID', 'Lat', 'Long']], on='ClusterID', how='left')
            summary = summary.rename(columns={'Lat': 'CentroidLat', 'Long': 'CentroidLong'})
        else:
            summary['CentroidLat'] = np.nan
            summary['CentroidLong'] = np.nan
        
        # Add StockPointID to the summary table
        summary['StockPointID'] = sp
        
        # Reorder columns in the summary table
        summary = summary[['StockPointID', 'ClusterID', 'NumCustomers', 'AvgDistance', 'CentroidLong', 'CentroidLat']]
        
        summary_list.append(summary)
    
    # Concatenate all assignments and summaries
    final_assignment = pd.concat(assignment_list, ignore_index=True)
    final_summary = pd.concat(summary_list, ignore_index=True)
    
    return final_assignment, final_summary


### Run Clustering

In [ ]:
prediction_output = pd.read_excel('../data/business_algo_output/recommendation_output.xlsx')
prediction_output = prediction_output.rename(columns={'FCID': 'StockPointID'})
customer_dim = pd.read_excel('../data/business_algo_output/customer_data.xlsx')
customer_dim = customer_dim.rename(columns={'Longitude': 'Long', 'Latitude': 'Lat'})

In [29]:
# Drop rows with missing coordinates or 0 long and lat
# Convert Lat and Long to numeric, handle invalid values
customer_dim[['Lat', 'Long']] = customer_dim[['Lat', 'Long']].apply(pd.to_numeric, errors='coerce')
customer_dim = customer_dim[(customer_dim['Long'] != 0) & (customer_dim['Lat'] != 0)]
customer_dim = customer_dim[customer_dim['Lat'].between(-90, 90)]
customer_dim = customer_dim[customer_dim['Long'].between(-180, 180)]
# Drop rows with NaN coordinates
customer_dim = customer_dim.dropna(subset=['Lat', 'Long']) 
 
# Drop rows with NaN coordinates
customer_dim = customer_dim.dropna(subset=['Lat', 'Long'])

In [30]:
# select only transactional customers with valid coordinates from transactional data
prediction_output = prediction_output[prediction_output['CustomerID'].isin(customer_dim['CustomerID'])]

In [31]:
print(prediction_output.columns.tolist())
print(customer_dim.columns.tolist())


['StockPointName', 'StockPointID', 'CustomerID', 'Category', 'SKUID', 'SKUCODE', 'rank_hist_cust_cat', 'L4M_n_orders', 'L4M_tot_qty', 'L4M_AOV', 'L4M_AOQ', 'L4M_Cust_Activity_SKU', 'L4M_Cust_Activity_Day_SKU', 'rank_hist_sp_cat', 'L4M_SP_Activity_SKU', 'rank_survey', 'SuggestType', 'SP_Product_Tag', 'RECOMMEND_FOR_PUSH']
['CustomerID', 'ContactName', 'BusinessName', 'CustomerModeName', 'ContactPhone', 'CustomerType', 'CustomerCreatedDate', 'Location', 'Address', 'FullAddress', 'StateName', 'CityName', 'TownID', 'TownName', 'Lat', 'Long', 'status', 'FirstName', 'LastName', 'IsLocationSubmitted', 'LocationSubmittedDate', 'IsLocationCaptured', 'IsLocationVerified', 'LocationVerifiedDate', 'RecaptureCount', 'CustomerStatus', 'RejectReason', 'RejectionDate']


In [32]:
# Example usage:
# Assuming prediction_output and customer_dim are your DataFrames
assignment_df, summary_df = cluster_customers_dbscan(prediction_output, customer_dim, eps_km=2.0, min_samples=3)
# print("Customer Assignments:")
# print(assignment_df)
# print("\nCluster Summaries:")
# print(summary_df)

In [36]:
summary_df

,StockPointID,ClusterID,NumCustomers,AvgDistance,CentroidLong,CentroidLat
0,1647081,0,7019,2.795363e+00,3.291009,6.587949
1,1647081,1,11,0.000000e+00,-78.865791,43.897093
2,1647081,2,10,0.000000e+00,3.076232,6.484355
3,1647081,3,30,9.563824e-01,6.592320,3.244106
4,1647081,4,10,0.000000e+00,3.345957,6.538340
...,...,...,...,...,...,...
68,1647113,37,46,9.366078e-01,6.456529,3.389495
69,1647113,38,10,0.000000e+00,3.312712,6.534257
70,1647113,39,48,0.000000e+00,4.877997,8.382902
71,1647113,40,5,8.819352e-14,3.430813,6.416737


## Route

In [38]:
def plan_routes(assignment_df, customer_dim, stock_point_dim):
    """
    Generate route plans for each cluster, starting and ending at the stock point.
    
    Args:
        assignment_df (pd.DataFrame): From Task 1, with StockPointID, CustomerID, ClusterID, SubclusterID
        customer_dim (pd.DataFrame): With CustomerID, Long, Lat
        stock_point_dim (pd.DataFrame): With StockPointID, Long, Lat
    
    Returns:
        tuple: (route_df, route_summary_df) containing route plans and summary of route distances
    """
    # Merge assignment_df with customer_dim to get customer coordinates
    route_data = assignment_df.merge(customer_dim[['CustomerID', 'Long', 'Lat']], on='CustomerID')
    
    # Validate customer coordinates
    route_data[['Long', 'Lat']] = route_data[['Long', 'Lat']].apply(pd.to_numeric, errors='coerce')
    route_data = route_data[route_data['Lat'].between(-90, 90) & route_data['Long'].between(-180, 180)]
    route_data = route_data.dropna(subset=['Long', 'Lat'])
    
    # Validate stock_point_dim coordinates
    stock_point_dim[['Long', 'Lat']] = stock_point_dim[['Long', 'Lat']].apply(pd.to_numeric, errors='coerce')
    stock_point_dim = stock_point_dim[stock_point_dim['Lat'].between(-90, 90) & stock_point_dim['Long'].between(-180, 180)]
    stock_point_dim = stock_point_dim.dropna(subset=['Long', 'Lat'])
    
    # Initialize lists for results
    route_list = []
    route_summary_list = []
    
    # Group by StockPointID and ClusterID
    groups = route_data.groupby(['StockPointID', 'ClusterID'])
    
    for (sp_id, cluster_id), group in groups:
        # Get stock point coordinates
        sp_coords = stock_point_dim[stock_point_dim['StockPointID'] == sp_id][['Long', 'Lat']]
        if sp_coords.empty:
            continue  # Skip if stock point not found
        sp_coord = (sp_coords['Lat'].iloc[0], sp_coords['Long'].iloc[0])
        
        # Get customer coordinates
        customers = group[['CustomerID', 'Long', 'Lat']].copy()
        if len(customers) == 0:
            continue
        
        # Initialize route with stock point
        route = [(sp_id, cluster_id, 'StockPoint', sp_coord[1], sp_coord[0], 0)]
        visited = set()
        current_coord = sp_coord
        total_distance = 0.0
        
        # Nearest Neighbor algorithm
        while len(visited) < len(customers):
            min_dist = float('inf')
            next_customer = None
            next_coord = None
            next_id = None
            
            # Find closest unvisited customer
            for _, row in customers.iterrows():
                if row['CustomerID'] not in visited:
                    cust_coord = (row['Lat'], row['Long'])
                    dist = geodesic(current_coord, cust_coord).kilometers
                    if dist < min_dist:
                        min_dist = dist
                        next_customer = row['CustomerID']
                        next_coord = cust_coord
                        next_id = row['CustomerID']
            
            if next_customer is None:
                break  # No more customers to visit
            
            # Add customer to route
            visited.add(next_customer)
            route.append((sp_id, cluster_id, next_id, next_coord[1], next_coord[0], len(visited)))
            total_distance += min_dist
            current_coord = next_coord
        
        # Return to stock point
        return_dist = geodesic(current_coord, sp_coord).kilometers
        total_distance += return_dist
        route.append((sp_id, cluster_id, 'StockPoint', sp_coord[1], sp_coord[0], len(visited) + 1))
        
        # Create route DataFrame
        route_df = pd.DataFrame(route, columns=['StockPointID', 'ClusterID', 'CustomerID', 'Long', 'Lat', 'RouteOrder'])
        route_list.append(route_df)
        
        # Create summary entry
        summary_df = pd.DataFrame({
            'StockPointID': [sp_id],
            'ClusterID': [cluster_id],
            'NumCustomers': [len(customers)],
            'TotalRouteDistance': [total_distance]
        })
        route_summary_list.append(summary_df)
    
    # Concatenate results
    final_route = pd.concat(route_list, ignore_index=True) if route_list else pd.DataFrame(columns=['StockPointID', 'ClusterID', 'CustomerID', 'Long', 'Lat', 'RouteOrder'])
    final_summary = pd.concat(route_summary_list, ignore_index=True) if route_summary_list else pd.DataFrame(columns=['StockPointID', 'ClusterID', 'NumCustomers', 'TotalRouteDistance'])
    
    return final_route, final_summary



In [39]:
# Example usage:
# Assuming assignment_df, customer_dim, and stock_point_dim are your DataFrames
route_df, route_summary_df = plan_routes(assignment_df, customer_dim, stock_point_dim)
print("Route Plans:")
print(route_df)
print("\nRoute Summaries:")
print(route_summary_df)

NameError: name 'stock_point_dim' is not defined